# Atop and Over in depth `⍤⍥`

[Atop](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Atop.htm), `⍤`, and [Over](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Over.htm), `⍥`, are compositional operators recently added to Dyalog. 

## Atop `f⍤g`

First atop, which has been assigned `function⍤function` thus sharing the symbol with the [rank](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Rank.htm) operator's `function⍤array`. You may be familiar with the 2-train, which is also called "atop": `(f g)Y` and `X(f g)Y`. Maybe you've even been burned by `f∘g Y` being an atop, but `X f∘g Y` _not_ being an atop. Well, the atop operator is what you would expect. In other words, `f⍤g Y` is exactly like `f∘g Y` but `X f⍤g Y` is `f X g Y` or `X (f g) Y`. We strongly recommend transitioning to use `⍤` in places where you've hitherto used monadic `f∘g`: it will prevent (at least one potential cause of) frustration if you ever decide to add a left argument to your code. 

Here's an example of what can happen. Let's say you define a function that returns the magnitude of reciprocal. 

In [1]:
|∘÷ ¯4
|∘÷ ¯5

0.25

0.2

(This could, of course, be written without the `∘` but we've opted for a very simple function for illustration purposes). Now you get a feature request that the function should take a left argument which is a numerator (instead of the default 1).

In [2]:
2 |∘÷ ¯4
2 |∘÷ ¯5

1.75

1.8

Oops. However:

In [3]:
|⍤÷ ¯4
|⍤÷ ¯5
2 |⍤÷ ¯4
2 |⍤÷ ¯5

0.25

0.2

0.5

0.4

One way to look at `f∘g` vs `f⍤g` is that when given a left argument, `∘` gives it to the left-hand function and `⍤` gives it to the right-hand function. Other than that, they are equivalent. Another way to look at `f∘g` vs `f⍤g` is simply choosing order of the first two tokens in the equivalent explicit expression: `X f∘g Y` computes `X f g Y` and `X f⍤g Y` computes `f X g Y`. So we're simply swapping `X` and `f`.

Then there's the classic problem with slashes, especially in tacit programming. If you've ever tried using replicate/compress in a train, you'll have bumped into the fact that slashes prefer being operators over being functions. This means that `{(5<⍵)/⍵}` _doesn't_ convert to `(5<⊢)/⊢`.

Whilst it may not be obvious at first sight, if we define `f←5<⊢` it might become clearer that `f/⊢` isn't at all what we want. Now, there's an axiom in APL that an operator cannot be an operand (shh, don't mention `∘.f`). This means that if a slash ends up in a situation where it has to be an operand, it will resort to being a function. You may even have noticed that constructs like `⊢(/⍨)5<⊢` work fine, but `⊢/⍨5<⊢` doesn't. This is because the `/` in isolation with the `⍨` is forced to become the operand of `⍨`. But since operators bind from the left, `⊢/` binds first, and so `⊢/⍨5<⊢` becomes `(⊢/)⍨5<⊢` or `(5<⊢)⊢/(5<⊢)` which is usually not what you want. 

So, `⍤` to the rescue.

If `⍤` (or any dyadic operator) is found to the immediate left of a slash, then clearly the dyadic operator cannot be the operand of the slash, `⍤` being a dyadic operator itself, and it can't be part of the function on the left, since it requires a right-operand, too. Therefore, the slash is forced to become a function.

So `-⍤/` is the negation of the replicate: 

In [4]:
1 0 2-⍤/10 20 30

¯10 ¯30 ¯30

It is easy to think then that "oh, this is an atop, so I should be able to do this with parentheses too; `(f g)`" but that'd be a mistake: `(-/)` is just a normal minus-reduction. Another mistake is to think: "if a slash is an operand, it'll be a function" and then think that `/∘⊢` would work like `⊢⍤/` by pre-processing the right argument with a no-op rather than post-processing the result with a no-op. 

Let's say we want a function `Apply` which takes an argument consisting of two vectors; a mask and some data, and computes `mask/data`. It isn't sufficient to write

In [5]:
Apply ← // ⍝ Incorrect; returns an enclosed result

Instead, we can write

In [6]:
Apply ← ⊃⊢⍤//

In fact, once `⊢⍤/` becomes more established as a common pattern, you can actually help the reader of your code by using `⊢⍤/` so they don't have to consider if your slash is replicate or reduce. For example, if your code says `z←x/y` it might not be obvious what's going on. If you instead write `z←x⊢⍤/y` your reader knows exactly what you're doing. 

Another somewhat contrived example: given an array, replace every element with two copies of itself prefixed and suffixed by the relevant prototypal element. E.g. `'abc'` becomes `' aa  bb  cc '`.  `10 20 30` should become `0 10 10 0 0 20 20 0 0 30 30 0`. 

In [9]:
f ← ⊢⊢⍤⍀⍨0 2 0⍴⍨3×≢
]display f 'abc'
f 10 20 30

┌→───────────┐
│ aa bb cc │
└────────────┘

0 10 10 0 0 20 20 0 0 30 30 0

Note how we force `⍀` into a function by saying `⊢⍤⍀` in the train.

## Over `⍥`

Remember how `f∘g` pre-processes the right argument of `f` using `g`? One way to look at [Over](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Over.htm) is simply as preprocessing _all_ arguments of `f` using `g`. All, as in both or the only. So, again `f⍥g Y` is the same as `f⍤g Y` and `f∘g Y`. The difference is, again, when we do a dyadic application. While `X f∘g Y` is `X f(g Y)` we have `X f⍥g Y` be `(g X)f(g Y)`. This may seem like an overly involved operator, but really, the pattern of pre-processing both arguments comes up a lot. Once you start looking for it, you'll see it all over.

For example, write a dyadic function computing the sum of absolute values of its arguments. 

In [10]:
¯5 +⍥| 23

28

Compare that to the old-fashioned `+/∘|`, or even `⊃(+/∘|{⍺⍵})`. Another: given arguments which are vectors, which one has the smallest maximum? Return ¯1 if the left argument has the smallest maximum, 1 if the right one has, or 0 if they are equal.

In [11]:
1 0 3 4 5 ×⍤-⍥(⌈/) 7 1 2 3 4 8 6 3

¯1

OK, how about this: write an alternative to replicate which can take arguments of equal shape, both with rank greater than 1, and replicates the corresponding elements. Since the result might otherwise be ragged, you have to return a vector. 

```apl
      (2 3⍴⍳6) YourFunction 2 3⍴⎕A
ABBCCCDDDDEEEEEFFFFFF
```

In [14]:
(2 3⍴⍳6) ⊢⍤/⍥, 2 3⍴⎕A ⍝ Replicate over catenate

ABBCCCDDDDEEEEEFFFFFF

In this case, you don't _need_ `⊢⍤` but it is good for clarity, and necessary if used inline in a train. One favourite is `,⍥⊂`prefer over {⍺ ⍵}